# Tugas 1 : Crawling Berita

**Pengertian Crawling**


Crawling adalah proses otomatis yang dilakukan oleh program komputer, yang disebut "crawler" atau "spider," untuk menelusuri dan mengumpulkan data dari situs web secara sistematis. Proses ini biasanya dimulai dengan mengakses halaman web tertentu dan kemudian mengikuti tautan (link) yang ada di dalamnya untuk mengakses halaman-halaman lain. Crawler mengunjungi halaman-halaman tersebut, mengumpulkan informasi seperti teks, gambar, dan metadata, dan menyimpan data tersebut untuk keperluan lebih lanjut, seperti pengindeksan di mesin pencari atau analisis data.

Crawling merupakan bagian penting dari proses web scraping dan pengindeksan mesin pencari, di mana mesin pencari seperti Google menggunakan crawler untuk mengumpulkan informasi dari seluruh web agar dapat menyediakan hasil pencarian yang relevan kepada pengguna.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, min_articles_per_category):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len([k for k in kategori_list if k == kategori]) >= min_articles_per_category:
            return  # Menghentikan proses jika jumlah artikel sudah mencapai minimum yang diinginkan

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = [
    "https://health.detik.com/berita-detikhealth/indeks",
    "https://travel.detik.com/travel-news/indeks"
]
categories = [
    "Kesehatan",
    "Pariwisata"
]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []
kategori_list = []

# Batas minimal artikel per kategori
min_articles_per_category = 50

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    page = 1
    while len([k for k in kategori_list if k == category]) < min_articles_per_category:
        url = f"{base_url}/{page}"
        get_data(url, category, min_articles_per_category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman
        page += 1

# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "tanggal": tanggal, "isi": isi, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)

# Menampilkan dataframe
print(df)

Dokter Ungkap Gejala Sinusitis Kronis, Dialami Ricky Harun hingga Masuk RS
Berkaca Kasus Viral Pria China, Ini Saran Dokter Biar Cabut Gigi Tak Berujung Fatal
Apa-apa Harus 'BPA Free', Yakin Pasti Aman?
Ternyata Otak Perempuan 'Menua' Lebih Cepat Pasca Pandemi COVID-19, Ini Risetnya
Profesor IPB Tegaskan 'Susu Ikan' Lebih Tepat Disebut Sari-Jus, Ini Katanya
Emi Shinohara, Pengisi Suara di Sailor Moon Meninggal Dunia karena Sakit
Ricky Harun Alami Sinusitis Kronis hingga Dirawat di RS, Kenali Pemicunya
Sederet Mitos Pemicu 'Paru-paru Basah', Pakai AC-Tidur di Lantai Termasuk?
5 Penangkal Kolesterol yang Ada di Dapur, Termasuk Jahe
Viral Diusulkan di Makan Bergizi Gratis Prabowo-Gibran, Apa Itu 'Susu Ikan'?
Kronologi Viral Pria di China Meninggal Dunia usai Cabut 23 Gigi dalam Sehari
Ricky Harun Dirawat di RS Akibat Sinusitis Kronis, Nyeri di Area Mata-Pipi
Ngeri, 62 Persen Kasus Mpox 'Ganas' di DR Kongo Serang Usia Anak
Ahli Gizi Sentil 'Susu Ikan' di Program Makan Bergizi Gratis Prabow

In [2]:
df=pd.read_csv("data_berita_detik.csv")
df

,judul,tanggal,isi,kategori
0,"Dokter Ungkap Gejala Sinusitis Kronis, Dialami...","Rabu, 11 Sep 2024 16:30 WIB",Jakarta - Ricky Harun baru-baru ini mengungkap...,Kesehatan
1,"Berkaca Kasus Viral Pria China, Ini Saran Dokt...","Rabu, 11 Sep 2024 16:04 WIB","Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tLantas, bo...",Kesehatan
2,"Apa-apa Harus 'BPA Free', Yakin Pasti Aman?","Rabu, 11 Sep 2024 15:24 WIB",Jakarta - Penggunaan produk dengan label 'BPA ...,Kesehatan
3,Ternyata Otak Perempuan 'Menua' Lebih Cepat Pa...,"Rabu, 11 Sep 2024 15:00 WIB",Jakarta - Otak dari remaja perempuan diketahui...,Kesehatan
4,Profesor IPB Tegaskan 'Susu Ikan' Lebih Tepat ...,"Rabu, 11 Sep 2024 14:31 WIB",Daftar Isi\n\n\n\n\r\n\t\t\t\t\t\t\tBagaimana ...,Kesehatan
...,...,...,...,...
95,"Bukannya Membantu, Petugas Bandara Jadi Maling...","Selasa, 10 Sep 2024 07:47 WIB","Tenerife, Spanyol - Bukannya membantu, sejumla...",Pariwisata
96,Angkasa Pura Indonesia Jadi Operator Bandara T...,"Selasa, 10 Sep 2024 06:42 WIB",Jakarta - Proses merger PT Angkasa Pura (AP) 1...,Pariwisata
97,Hotel Menjadi-jadi di Bali: Pengamat: Moratori...,"Selasa, 10 Sep 2024 06:01 WIB",Jakarta - Bali selatan sudah sangat jenuh deng...,Pariwisata
98,"Hotel di Mandalika Mahal Menjelang MotoGP, Ini...","Selasa, 10 Sep 2024 05:33 WIB",Jakarta - Lombok kembali jadi tuan rumah perge...,Pariwisata
